In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import matplotlib
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime as dt
import sklearn.metrics as metrics
import pickle

import sys,random
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, f1_score, log_loss, confusion_matrix
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB

In [ ]:
def implt(img):
    plt.figure()
    plt.imshow(img)
    plt.axis('off')

print(implt)

plt.style.use('seaborn')                   # if want to use the default style, set 'classic'
plt.rcParams['ytick.right']     = True
plt.rcParams['ytick.labelright']= True
plt.rcParams['ytick.left']      = False
plt.rcParams['ytick.labelleft'] = False
plt.rcParams['figure.figsize']  = [7,7]   # Set the figure size to be 7 inch for (width,height)

print("Matplotlib setup completes.")

<function implt at 0x7fbd78cd0b90>
Matplotlib setup completes.


In [ ]:
DIR = '/content/gdrive/My Drive/iss/covid/data/'

import pickle 

X_train = pickle.load(open(DIR+"X_train_probaML_random21.sav", 'rb'))
X_test = pickle.load(open(DIR+"X_test_probaML_random21.sav", 'rb'))
y_train = pickle.load(open(DIR+"y_train_probaML_random21.sav", 'rb'))
y_test = pickle.load(open(DIR+"y_test_probaML_random21.sav", 'rb'))
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(366577, 63)
(366577,)
(91645, 63)
(91645,)


In [ ]:
X = np.concatenate((X_train,X_test),axis=0)[:,:59]
Y = np.concatenate((y_train,y_test),axis=0)

in every iteration: 
1. Cell 1 parameter are updated to the current best value
2. Cell 2 param_to_opt are changed accordingly (parameter and value)
3. repeated for all the tuneable hyperparameter


## Light gbm

In [ ]:
# CELL 1
LC = lgb.LGBMClassifier(max_depth=17, num_leaves=40, feature_fraction=0.7, subsample=0.75, boosting='dart', lambda_l1=0.01, learning_rate=0.1, 
                        random_state = 21,objective= 'binary', metric= 'auc', is_unbalance= True, bagging_freq=5, num_boost_round=300,early_stopping_rounds=30 )


In [ ]:
# CELL 2

param_to_opt_1 = {
 'learning_rate': [0.0001,0.001,0.01,0.1],
}
grid_search_1 = GridSearchCV(estimator = LC,
                             param_grid = param_to_opt_1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
grid_search_1.fit(X,Y)
grid_search_1.best_params_, grid_search_1.best_score_


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


({'learning_rate': 0.1}, 0.9272300818809459)

# XGB

In [ ]:
# CELL 1
xgb = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=11,
     min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
     objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=21,use_label_encoder=False, eval_metric='auc')

In [ ]:
# CELL 2
param_to_opt_1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
grid_search_1 = GridSearchCV(estimator = xgb, param_grid = param_to_opt_1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
grid_search_1.fit(X_train,y_train)
grid_search_1.best_params_, grid_search_1.best_score_


# RFC

In [ ]:
# CELL 1
RFC = RandomForestClassifier(n_estimators=500, min_samples_split=2, max_depth=30, 
                            criterion='entropy', random_state=21, class_weight={0:1,1:1},
                            max_features='log2')

In [ ]:
# CELL 2
param_to_opt_1 = {
 'n_estimators':range(100,1000,100),
}
grid_search_1 = GridSearchCV(estimator = RFC, param_grid = param_to_opt_1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
grid_search_1.fit(X_train,y_train)
grid_search_1.best_params_, grid_search_1.best_score_


## LR CLASSIFIER

In [ ]:
# CELL 1
lr = LogisticRegression(C=1.0, intercept_scaling=1,   
               dual=False, fit_intercept=False, penalty='l2', tol=0.0001)


In [ ]:
# CELL 2
param_to_opt_1 = {
 'C': [0.0001, 0.001, 0.01, 0.1, 1]
}
grid_search_1 = GridSearchCV(estimator = lr, param_grid = param_to_opt_1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
grid_search_1.fit(X_train,y_train)
grid_search_1.best_params_, grid_search_1.best_score_
